In [1]:
# this file combare the result between SEN_model and Triplet_LSTM_model
# by using different SongParts which filted form [3,10] in the sameSong
# remeber to normalize the song part

In [2]:
import numpy as np
import librosa
import glob
import os
from random import randint
import torch
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import matplotlib.pyplot as plt
%matplotlib inline

import import_ipynb

In [3]:
#randome filter the song in such parts
songPart = [3,4,5,6,7,8,9,10]
#when part=10，the orderation is very big. 
#so wo test 100 song rather than more than 100song. the premutaion is 100*(10*9*8) pairs
sample = 100

In [4]:
torch.cuda.set_device(3)
print(torch.cuda.current_device())

3


In [5]:
class SEN(nn.Module):
    def __init__(self):
        super().__init__()
        self.siamese_cnn = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=[4,128],stride=[1,128]),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 256, kernel_size=[4,1],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.Conv2d(256, 256, kernel_size=[4,1],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            )
        self.late_cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=[3,3],stride=[1,1]),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=[3,3],stride=[3,3],padding=1),
            
            nn.Conv2d(64, 128, kernel_size=[3,3],stride=[1,1]),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=[3,3],stride=[3,3],padding=[1,0]),

            nn.Conv2d(128, 256, kernel_size=[3,3],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            )
        self.fcWithDropout = nn.Sequential(
            nn.Linear(768, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            
            nn.Linear(1024, 2),
            nn.Softmax(dim=1)
            )
    def cal_similarity(self,matrix1,matrix2):
        out1 = torch.squeeze(matrix1,dim=3)
        out2 = torch.squeeze(matrix2,dim=3)
        num = torch.bmm(torch.transpose(out1,1,2),out2)
        h1_norm = torch.sqrt(torch.sum(torch.mul(out1,out1), dim=1, keepdim=True))
        h2_norm = torch.sqrt(torch.sum(torch.mul(out2,out2), dim=1, keepdim=True))
        denom = torch.bmm(torch.transpose(h1_norm,1,2),h2_norm)
        fms = torch.unsqueeze(torch.div(num,denom),dim=1)
        return fms
    
    def _reduce_var(self,inputs):
        m1 = torch.mean(inputs,dim=2,keepdim=True)
        m = torch.mean(m1,dim=3,keepdim=True)
        devs_squared1 = torch.mul(inputs - m,inputs - m)
        devs_squared2 = torch.mean(devs_squared1,dim=2)
        devs_squared = torch.mean(devs_squared2,dim=2)
        return devs_squared

    def cal_global_pool(self,matrix):
        g_max1,_ = torch.max(matrix,dim=2)
        g_max,_ = torch.max(g_max1,dim=2)
        g_mean1 = torch.mean(matrix,dim=2)
        g_mean = torch.mean(g_mean1,dim=2)
        g_var = self._reduce_var(matrix)
        return torch.cat([g_max, g_mean, g_var], 1)
        
    def forward(self,x1,x2):
        out1 = self.siamese_cnn(x1)
        out2 = self.siamese_cnn(x2)
        similarity = self.cal_similarity(out1,out2)
        late_cnn_out = self.late_cnn(similarity)
        golbal_pool_out = self.cal_global_pool(late_cnn_out)
        predictions = self.fcWithDropout(golbal_pool_out)
        return predictions

In [6]:
class TripletLSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self.siamese_cnn = nn.Sequential(
            nn.Conv2d(1, 128, kernel_size=[4,128],stride=[1,128]),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 256, kernel_size=[4,1],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, kernel_size=[4,1],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            )
        #max_pool need keep input.shape=output.shape
        self.late_cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=[3,3],stride=[1,1]),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=[3,3],stride=[3,3],padding=1),

            nn.Conv2d(64, 128, kernel_size=[3,3],stride=[1,1]),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=[3,3],stride=[3,3],padding=[1,0]),

            nn.Conv2d(128, 256, kernel_size=[3,3],stride=[1,1]),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            )
        self.fcWithDropout = nn.Sequential(
            nn.Linear(256, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),

            nn.Linear(1024, 2),
            nn.Softmax(dim=1),
            )
        self.rnnLSTM = nn.LSTM(
            input_size=256, 
            hidden_size=128,
            num_layers=2,
            dropout=0.1,
            batch_first=True,
            bidirectional=True,
            )
        
    def cal_similarity(self,matrix1,matrix2):
        out1 = torch.squeeze(matrix1,dim=3)
        out2 = torch.squeeze(matrix2,dim=3)
        num = torch.bmm(torch.transpose(out1,1,2),out2)
        h1_norm = torch.sqrt(torch.sum(torch.mul(out1,out1), dim=1, keepdim=True))
        h2_norm = torch.sqrt(torch.sum(torch.mul(out2,out2), dim=1, keepdim=True))
        denom = torch.bmm(torch.transpose(h1_norm,1,2),h2_norm)
        fms = torch.unsqueeze(torch.div(num,denom),dim=1)
        return fms
    
    def _reduce_var(self,inputs):
        m1 = torch.mean(inputs,dim=2,keepdim=True)
        m = torch.mean(m1,dim=3,keepdim=True)
        devs_squared1 = torch.mul(inputs - m,inputs - m)
        devs_squared2 = torch.mean(devs_squared1,dim=2)
        devs_squared = torch.mean(devs_squared2,dim=2)
        return devs_squared
    
    def cal_global_pool(self,matrix):
        g_max1,_ = torch.max(matrix,dim=2)
        g_max,_ = torch.max(g_max1,dim=2)
        g_mean1 = torch.mean(matrix,dim=2)
        g_mean = torch.mean(g_mean1,dim=2)
        g_var = self._reduce_var(matrix)
        return torch.cat([g_max, g_mean, g_var], 1)
    
    def cal_lstm(self,seqInput):
        #view batchsize seq-length input-size
        #dataParaller may change the batchsize
        lstm_batchSize = list(seqInput.size())[0]
        seqInput = seqInput.view(lstm_batchSize,3,-1)
        outLstm,(_,_) = self.rnnLSTM(seqInput)
        return outLstm[:,-1,:]
        
    def forward(self,x1,x2,x3):
        out1 = self.siamese_cnn(x1)
        out2 = self.siamese_cnn(x2)
        out3 = self.siamese_cnn(x2)
        out4 = self.siamese_cnn(x3)
        similarity1 = self.cal_similarity(out1,out2)
        similarity2 = self.cal_similarity(out3,out4)
        late_cnn_out = self.late_cnn((similarity1 + similarity2)/2)
        golbal_pool_out = self.cal_global_pool(late_cnn_out)
        lstmout = self.cal_lstm(golbal_pool_out)
        predictions = self.fcWithDropout(lstmout)
        return predictions

In [8]:
tripleNet_LSTM_model = TripletLSTM().double().cuda()
state_dict = torch.load('./model/tripleNet_LSTM.pkl',map_location={'cuda:0':'cuda:3'})
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    name = k[7:] 
    new_state_dict[name] = v
tripleNet_LSTM_model.load_state_dict(new_state_dict)
tripleNet_LSTM_model = tripleNet_LSTM_model.eval()

#test about 100 songs in 10 part ，so later we use in different parts test
#100*10*9*8
LSTM_testdata = []
LSTM_testlabel = []

#所有的triplet的总数   1000*10*9*8
for song in range(sample):
    for i in range(10):
        for j in range(10):
            for k in range(10):
                if(i != j and j != k and i !=k):
                    triplet = "song{}_{},song{}_{},song{}_{}".format(song,i,song,j,song,k)
                    LSTM_testdata.append(triplet)
                if(j-i==1 and k-j==1):
                    LSTM_testlabel.append(0)
                elif(i != j and j != k and i !=k):
                    LSTM_testlabel.append(1)
                    
print(len(LSTM_testdata))
print(len(LSTM_testlabel))


72000
72000


In [9]:
SENmodel = SEN().double().cuda()
state_dict = torch.load('./model/baseline_sen.pkl',map_location={'cuda:2':'cuda:3'})
SENmodel.load_state_dict(state_dict)
SENmodel = SENmodel.eval()

#test about 100 songs in 10 part ，so later we use in different parts test
#100*10*9
SEN_testdata = []
SEN_testlabel = []

for song in range(sample):
    for i in range(10):
        for j in range(10):
                if(i != j):
                    test_pair = "song{}_{},song{}_{}".format(song,i,song,j)
                    SEN_testdata.append(test_pair)
                if(j-i==1):
                    SEN_testlabel.append(0)
                elif(i != j):
                    SEN_testlabel.append(1)
print(len(SEN_testdata))
print(len(SEN_testlabel))

9000
9000


In [10]:
data_dir = "/media/data/cuixuange/ScrawlMusic/train_data_10part_normalizatin/"
def load_exits_file(train_data,index):
    str_data = train_data[index].split(",")      
    x1_name = data_dir + str_data[0]+".npy"
    x2_name = data_dir + str_data[1]+".npy"
    x3_name = data_dir + str_data[2]+".npy"
    if os.path.isfile(x1_name) and os.path.isfile(x2_name) and os.path.isfile(x3_name): 
        return x1_name,x2_name,x3_name,index
    else:
        index += 6
        return load_exits_file(train_data,index)
    

result_LSTM = {}
result = []
print(len(LSTM_testdata))


with torch.no_grad():
    for key,token in enumerate(LSTM_testdata):
        x1_name,x2_name,x3_name,exit_index = load_exits_file(LSTM_testdata,key)
        x1 = torch.from_numpy(np.load(x1_name)).unsqueeze(0).unsqueeze(0).cuda()
        x2 = torch.from_numpy(np.load(x2_name)).unsqueeze(0).unsqueeze(0).cuda()
        x3 = torch.from_numpy(np.load(x3_name)).unsqueeze(0).unsqueeze(0).cuda()

        predict_label = tripleNet_LSTM_model(x1,x2,x3).detach().cpu()
        result_LSTM[token] = predict_label[0][0]
        
                
#         if(key == 100):
#             break

print(len(result_LSTM))

72000
72000


In [11]:
data_dir = "/media/data/cuixuange/ScrawlMusic/train_data_10part_normalizatin/"
def load_exits_file(train_data,index):
    str_data = train_data[index].split(",")      
    x1_name = data_dir + str_data[0]+".npy"
    x2_name = data_dir + str_data[1]+".npy"
    if os.path.isfile(x1_name) and os.path.isfile(x2_name):
        return x1_name,x2_name,index
    else:
        index += 6
        return load_exits_file(train_data,index)
    

result_SEN = {}
result = []
print(len(SEN_testdata))

with torch.no_grad():
    for key,token in enumerate(SEN_testdata):
        x1_name,x2_name,exit_index = load_exits_file(SEN_testdata,key)
        X1 = torch.from_numpy(np.expand_dims(np.transpose(np.load(x1_name)),axis=2))
        X2 = torch.from_numpy(np.expand_dims(np.transpose(np.load(x2_name)),axis=2))
        X = torch.cat((X1,X2),dim=2)
        X = X.unsqueeze(0)
        X = X.unsqueeze(0) 
        x1 =torch.transpose((X[...,0]),2,3).cuda()
        x2 =torch.transpose((X[...,1]),2,3).cuda()

        predict_label = SENmodel(x1,x2).detach().cpu()
        result_SEN[token] = predict_label[0][0]
        
#         if(key == 100):
#             break

print(len(result_SEN))

9000
9000


In [12]:
from sympy.utilities.iterables import multiset_permutations

def getMaxScoreInSong(index,songPart,modelName):
    """
    input:  the index og the song, 
            the part(int) such as 3 or 4 ....or 10
            which model to calculate
    output: finalsorce, the sort which has max socre
    """
    ps = [p for p in multiset_permutations(np.arange(songPart))]
    maxSocre = 0.0
    maxArrangement = []
    for p in ps:
        score = 0.0
        if(modelName == "TripletLSTM"):
            score = getLSTM_Score(index,p,songPart)
        if(modelName == "SEN"):
            score = getSEN_Score(index,p,songPart)
        if maxSocre < score:
            maxSocre = score
            maxArrangement = p
    return maxSocre,maxArrangement

def isCorrectArrangement(listP):
    """
    Judge is correct order，such as 1,2,3,4 is correct order
    """
    for i in range(len(listP)-2):
        if(listP[i+1] - listP[i] != 1):
            return False
    return True

def getSEN_Score(index,listPart,songPart):
    """
    input: index of this song
           listpart is current order which is waiting to calculate
           songPart is the parts of this song  
    output:current listPart socre
    """
    token = []
    score = 0.0
    for i in range(0,songPart-1):
        pair = "song{}_{},song{}_{}".format(index,listPart[i],index,listPart[i+1])
        token.append(pair)
    for i in token:
        score += result_SEN[i]
    return score

def getLSTM_Score(index,listPart,songPart):
    """
    input: index of this song
           listpart is current order which is waiting to calculate
           songPart is the parts of this song  
    output:current listPart socre
    """
    token = []
    score = 0.0
    for i in range(0,songPart-2):
        triplet = "song{}_{},song{}_{},song{}_{}".format(index,listPart[i],index,listPart[i+1],index,listPart[i+2])
        token.append(triplet)
    for i in token:
        score += result_LSTM[i]
    return score

In [13]:
from __future__ import division

def calGA_Accuracy(songPart,modelName):
    """
    songPart:  parts of song to order by
    modelName: which model to use
    """
    import time
    start = time.time()
    count = 0
    for index in range(sample):
        _,arrange = getMaxScoreInSong(index,songPart,modelName)
        if(isCorrectArrangement(arrange) == True):
            count += 1     
    end = time.time()
    print("modelName={}\tPart={}\tAccuracy={}\ttime={}\t".format(modelName,songPart,count/sample,end - start))

for part in  songPart:
    calGA_Accuracy(part,"SEN")
    calGA_Accuracy(part,"TripletLSTM")
#     if(part==4):
#         break

modelName=SEN	Part=3	Accuracy=0.67	time=0.05901932716369629	
modelName=TripletLSTM	Part=3	Accuracy=0.79	time=0.04198050498962402	
modelName=SEN	Part=4	Accuracy=0.43	time=0.08721303939819336	
modelName=TripletLSTM	Part=4	Accuracy=0.69	time=0.06798338890075684	
modelName=SEN	Part=5	Accuracy=0.32	time=0.25928759574890137	
modelName=TripletLSTM	Part=5	Accuracy=0.53	time=0.24993038177490234	
modelName=SEN	Part=6	Accuracy=0.17	time=1.1610150337219238	
modelName=TripletLSTM	Part=6	Accuracy=0.44	time=1.068131446838379	
modelName=SEN	Part=7	Accuracy=0.11	time=8.835685968399048	
modelName=TripletLSTM	Part=7	Accuracy=0.37	time=8.978431463241577	
modelName=SEN	Part=8	Accuracy=0.05	time=78.84110617637634	
modelName=TripletLSTM	Part=8	Accuracy=0.32	time=81.14716219902039	
modelName=SEN	Part=9	Accuracy=0.03	time=778.9665851593018	
modelName=TripletLSTM	Part=9	Accuracy=0.29	time=826.0625054836273	
modelName=SEN	Part=10	Accuracy=0.0	time=8769.560252666473	
modelName=TripletLSTM	Part=10	Accuracy=0.24	ti